In [10]:
import numpy as np
from dt import *
import pydot 

train_set=np.load('dt/train_set.npy')
train_labels=np.load('dt/train_labels.npy')
test_set=np.load('dt/test_set.npy')
test_labels=np.load('dt/test_labels.npy')

In [11]:
#Given features and number of classes
numerical_features = 4
num_classes = 3


In [12]:
#Create a class to fill tree
class Node:
    name = None
    left_child = None
    right_child = None
    bucket = None
    calculated_split_value = None
    num_attribute = None
    


In [13]:
#Basic for loop to calculate bucket.
def calculate_bucket(label,num_classes):
    bucket = []
    for i in range(num_classes):
        bucket.append((label == i).sum())
    return bucket

In [14]:
#Basic accuracy function.
def calculate_accuracy(prediction,test_labels):
    accuracy = np.subtract(prediction,test_labels)
    return accuracy

In [15]:

#Creating tree
def make_dt(train_set,train_labels,parent_node,heuristic_name):
    #Deciding function according to heuristic_name
    fnction = np.argmax
    if(heuristic_name == 'avg_gini_index'):
        fnction = np.argmin

    parent_node.bucket = calculate_bucket(train_labels,num_classes) 
    #Checking stop condition
    if(heuristic_name == 'avg_gini_index'):
        compare = gini(parent_node.bucket)
        if(compare  == 0):
            return 
    else:
        compare = entropy(parent_node.bucket)
        if(compare  == 0):
            return 
    results_value = []
    results_split_value = []
    #Checking which feature is best according to gini and info and what is the split value.
    for i in range(numerical_features):
        result = calculate_split_values(train_set, train_labels, num_classes, i, heuristic_name)
        mins = fnction(result[:,1],axis=0)
        results_value.append(result[:,1][mins])
        results_split_value.append(result[:,0][mins])
    #Giving that values.
    parent_node.num_attribute = fnction(results_value)
    parent_node.calculated_split_value = results_split_value[parent_node.num_attribute]

    count = 0
    while(len(train_set) == 2 and train_set[0][parent_node.num_attribute] == train_set[1][parent_node.num_attribute]):      
        parent_node.num_attribute = count
        parent_node.calculated_split_value = results_split_value[parent_node.num_attribute]
        count +=1
    #Divide label and data
    left_train_set = train_set[train_set[:,parent_node.num_attribute] < parent_node.calculated_split_value]
    right_train_set = train_set[train_set[:,parent_node.num_attribute] >= parent_node.calculated_split_value]
    left_label = train_labels[train_set[:,parent_node.num_attribute] < parent_node.calculated_split_value]
    right_label = train_labels[train_set[:,parent_node.num_attribute] >= parent_node.calculated_split_value]
    #Call function again
    left_child = Node()
    right_child = Node()
    left_child.name = parent_node.name*2
    right_child.name = parent_node.name*2+1
    parent_node.left_child = left_child
    parent_node.right_child = right_child
    #Some stop conditions 
    if(len(left_label) == 0):
        return 
    make_dt(left_train_set,left_label,parent_node.left_child,heuristic_name)
    if(len(right_label) == 0):
        return
    make_dt(right_train_set,right_label,parent_node.right_child,heuristic_name)

    
    
   

In [16]:
#Basic test function. If you find the leaf node get the max argument label in that bucket. Otherwise continue to left right according to split value.
def test_DT(root,test_set,prediction):
    if(root.calculated_split_value == None):
        
        prediction.append(np.argmax(root.bucket))
        return
    
    if(root.calculated_split_value > test_set[root.num_attribute]):
        test_DT(root.left_child,test_set,prediction)
    else:
        test_DT(root.right_child,test_set,prediction)
    

In [17]:
#Creating tree with pydot. It is basic tree traversal like in C.
def print_DT(root,graph):
    if(root.left_child != None):
        print_DT(root.left_child,graph)
    if(root.right_child != None):
        print_DT(root.right_child,graph)

    node_name = "x["+str(root.num_attribute)+"]<"+str(root.calculated_split_value)+"\n"+str(root.bucket)
    if(root.calculated_split_value == None):
        node_name = str(root.bucket)
    node = pydot.Node(root.name,label=node_name)
    graph.add_node(node)

    if(root.left_child != None):
        print_DT(root.left_child,graph)
    if(root.right_child != None):
        print_DT(root.right_child,graph)
    if(root.calculated_split_value != None):
        edge1 = pydot.Edge(str(root.name),str(root.name*2),label="<")
        edge2 = pydot.Edge(str(root.name),str(root.name*2+1),label=">=")
        graph.add_edge(edge1)
        graph.add_edge(edge2)

In [18]:
#For two different function create tree print tree and test tree. It prints accuracy and also write trees to a png file.
h_name = ['info_gain','avg_gini_index']
for i in h_name:
    parent_node = Node()
    parent_node.name = 1
    make_dt(train_set,train_labels,parent_node,i)
    graph = pydot.Dot(graph_type='digraph',strict=True)
    print_DT(parent_node,graph)
    out_name = i + ".png"
    graph.write_png(out_name)
    prediction = []
    for i in range(len(test_set)):
        test_DT(parent_node,test_set[i],prediction)
    accuracy = calculate_accuracy(prediction,test_labels)
    accuracy = np.count_nonzero(accuracy == 0)/len(accuracy)*100
    print(accuracy)

96.66666666666667
90.0


In [19]:
#Chi table according to degree of freedom. Same steps are implemented at following. Just chi square check added.
chi_table = [2.706, 4.605, 6.251]

In [20]:
def make_dt_chi_square(train_set,train_labels,parent_node,heuristic_name):
    fnction = np.argmax
    if(heuristic_name == 'avg_gini_index'):
        fnction = np.argmin

    parent_node.bucket = calculate_bucket(train_labels,num_classes) 
    if(heuristic_name == 'avg_gini_index'):
        compare = gini(parent_node.bucket)
        if(compare  == 0):
            return 
    else:
        compare = entropy(parent_node.bucket)
        if(compare  == 0):
            return 
    results_value = []
    results_split_value = []

    for i in range(numerical_features):
        result = calculate_split_values(train_set, train_labels, num_classes, i, heuristic_name)
        mins = fnction(result[:,1],axis=0)
        results_value.append(result[:,1][mins])
        results_split_value.append(result[:,0][mins])
    
    parent_node.num_attribute = fnction(results_value)
    parent_node.calculated_split_value = results_split_value[parent_node.num_attribute]

    count = 0
    while(len(train_set) == 2 and train_set[0][parent_node.num_attribute] == train_set[1][parent_node.num_attribute]):      
        parent_node.num_attribute = count
        parent_node.calculated_split_value = results_split_value[parent_node.num_attribute]
        count +=1

    left_train_set = train_set[train_set[:,parent_node.num_attribute] < parent_node.calculated_split_value]
    right_train_set = train_set[train_set[:,parent_node.num_attribute] >= parent_node.calculated_split_value]
    left_label = train_labels[train_set[:,parent_node.num_attribute] < parent_node.calculated_split_value]
    right_label = train_labels[train_set[:,parent_node.num_attribute] >= parent_node.calculated_split_value]
    #After separate the labels check if chi square reject or accept it. If condition is true, we are returning without splitting data.
    chi_value, degree_freedom = chi_squared_test(calculate_bucket(left_label,num_classes) ,calculate_bucket(right_label,num_classes))
    if chi_value <= chi_table[degree_freedom-1]:
        return
    
    left_child = Node()
    right_child = Node()
    left_child.name = parent_node.name*2
    right_child.name = parent_node.name*2+1
    parent_node.left_child = left_child
    parent_node.right_child = right_child
    if(len(left_label) == 0):
        return 
    make_dt_chi_square(left_train_set,left_label,parent_node.left_child,heuristic_name)
    if(len(right_label) == 0):
        return
    make_dt_chi_square(right_train_set,right_label,parent_node.right_child,heuristic_name)

In [21]:
#Doing test as same way. Just added extra if conditions according to leaf node changes.
def test_DT_chi(root,test_set,prediction):
    if(root.calculated_split_value == None or (root.left_child == None and root.right_child == None)):    
        prediction.append(np.argmax(root.bucket))
        return
    
    if(root.calculated_split_value > test_set[root.num_attribute]):
        if(root.left_child == None):
            return
        test_DT_chi(root.left_child,test_set,prediction)
    else:
        if(root.right_child == None):
            return
        test_DT_chi(root.right_child,test_set,prediction)

In [22]:
#Same with previous print tree. Just added extra if conditions according to leaf node changes.
def print_DT_chi(root,graph):
    
    if(root.left_child != None):
        print_DT_chi(root.left_child,graph)
    if(root.right_child != None):
        print_DT_chi(root.right_child,graph)

    node_name = "x["+str(root.num_attribute)+"]<"+str(root.calculated_split_value)+"\n"+str(root.bucket)
    
    if(root.calculated_split_value == None):
        node_name = str(root.bucket)
    node = pydot.Node(root.name,label=node_name)
    graph.add_node(node)

    if(root.left_child != None):
        print_DT_chi(root.left_child,graph)
    if(root.right_child != None):
        print_DT_chi(root.right_child,graph)
    #This if added to check if it is leaf node or not. Calculated split value check doesnt enough anymore.
    if(root.left_child == None and root.right_child == None):
        return
    if(root.calculated_split_value != None):
        edge1 = pydot.Edge(str(root.name),str(root.name*2),label="<")
        edge2 = pydot.Edge(str(root.name),str(root.name*2+1),label=">=")
        graph.add_edge(edge1)
        graph.add_edge(edge2)

In [23]:
#For two different function create tree print tree and test tree. It prints accuracy and also write trees to a png file with the name chi on them.
h_name = ['info_gain','avg_gini_index']
for i in h_name:
    parent_node = Node()
    parent_node.name = 1
    make_dt_chi_square(train_set,train_labels,parent_node,i)
    graph = pydot.Dot(graph_type='digraph',strict=True)
    print_DT_chi(parent_node,graph)
    out_name = i + "_chi.png"
    graph.write_png(out_name)
    prediction2 = []
    for i in range(len(test_set)):
        test_DT_chi(parent_node,test_set[i],prediction2)
    accuracy = calculate_accuracy(prediction2,test_labels)
    accuracy = np.count_nonzero(accuracy == 0)/len(accuracy)*100
    print(accuracy)

96.66666666666667
90.0
